In [ ]:
# this one is too slow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained("stabilityai/StableBeluga-7B", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("stabilityai/StableBeluga-7B", torch_dtype=torch.float32, low_cpu_mem_usage=True, device_map="auto", offload_folder="offload_folder")
system_prompt = "### System:\nYou are StableBeluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.\n\n"

message = "Write me a poem please"
prompt = f"{system_prompt}### User: {message}\n\n### Assistant:\n"
inputs = tokenizer(prompt, return_tensors="pt")#.to("cuda")
output = model.generate(**inputs, do_sample=True, top_p=0.95, top_k=0, max_new_tokens=256)

print(tokenizer.decode(output[0], skip_special_tokens=True))

# RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from
# but I want to use CPU
# instead call the model like this:
model = AutoModelForCausalLM.from_pretrained("stabilityai/StableBeluga-7B", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")

In [ ]:
# %%html
# <style>
# .cell-output-ipywidget-background {
#    background-color: transparent !important;
# }
# .jp-OutputArea-output {
#    background-color: transparent;
# }  
# </style>

In [ ]:
# import ipywidgets as widgets
# from IPython.display import display

# text = widgets.Textarea(
#     value='Hello World',
#     description='String:',
#     layout=widgets.Layout(width="50%", height="300px"),
# )
# display(text)

In [ ]:
def ave_hook(resid_pre, hook):
    coeff = 5
    if resid_pre.shape[1] == 1:
        return  # caching in model.generate for new tokens
    # We only add to the prompt (first call), not the generated tokens.
    ppos, apos = resid_pre.shape[1], act_diff.shape[1]
    assert apos <= ppos, f"More mod tokens ({apos}) then prompt tokens ({ppos})!"
    # add to the beginning (position-wise) of the activations
    resid_pre[:, :apos, :] += coeff * act_diff

act_name = 6
editing_hooks = [(f"blocks.{act_name}.hook_resid_pre", ave_hook)]
with model.hooks(fwd_hooks=editing_hooks):
    # tokenized = model.to_tokens(prompt_batch)
    # r = model.generate(input=tokenized, max_new_tokens=20, do_sample=True, **kwargs)
    new_text = model.generate(text.value, max_new_tokens=1, temperature=1, verbose=False)
    text.value = new_text


In [ ]:
def get_dims(resid_pre, hook):
    print(resid_pre.shape, hook)
    # resid_pre[:, -1, :] *= 0.5
    # get norm of this vector
    norm = torch.norm(resid_pre[:, -1, :], dim=-1, keepdim=True)
    print(norm)
    print(resid_pre)


act_name = 6
editing_hooks = [(f"blocks.{act_name}.hook_resid_pre", get_dims)]
with model.hooks(fwd_hooks=editing_hooks):
    # tokenized = model.to_tokens(prompt_batch)
    # r = model.generate(input=tokenized, max_new_tokens=20, do_sample=True, **kwargs)
    new_text = model.generate("this is some", max_new_tokens=10, temperature=1, verbose=False)
    # text.value = new_text
    print(new_text)
